# Using the Script-Language Container

A [Script-Language Container](https://github.com/exasol/script-languages-release) for the Exasol database consists of a Linux container with a complete Linux distribution and all required libraries, such as a script client. The script client is responsible for the communication with the database and for executing the script code. It allows to also include user specific libraries which can then be used from within the UDF's.

## Prerequisites

To run this Notebook you need:
- Jupyter with Python3.6+ Kernel
- Docker 17.05+
- git
- Your Notebook user needs permissions to run Docker

## Preparing the Notebook

First we need to install and import a few Python packages required in the course of this Notebook.

In [3]:
!pip install -r requirements.txt

In [6]:
import bash_runner as bash # A helper to run bash with interactive output from python
import importlib
from pathlib import Path
import pyexasol
import requests
import textwrap

## Cloning the Git Repository

To use the [Script-Language Container](https://github.com/exasol/script-languages-release) we need to clone the Git Repository with

```
git clone https://github.com/exasol/script-languages-release --recursive
```

We need to use `--recursive` to also clone the sub-modules of the repository.

**Note:**You can use the following code snippet, that either clones the repository if it isn't already cloned or resets it to the current origin/master branch, such that we have always a defined state for the remaining Notebook.

In [7]:
slc_path="script-languages-release"
if not Path(slc_path).exists():
    bash.run("""
    git clone https://github.com/exasol/script-languages-release --recursive
    """)
else:
    bash.run(f"""
    cd {slc_path}
    git fetch
    git reset --hard origin/master 
    git submodule foreach git reset --hard origin/master 
    """)

Cloning into 'script-languages-release'...
remote: Enumerating objects: 7350, done.
remote: Counting objects: 100% (1346/1346), done.
remote: Compressing objects: 100% (761/761), done.
remote: Total 7350 (delta 696), reused 1178 (delta 562), pack-reused 6004
Receiving objects: 100% (7350/7350), 1.12 MiB | 22.00 MiB/s, done.
Resolving deltas: 100% (3975/3975), done.
Submodule 'script-languages' (https://github.com/exasol/script-languages.git) registered for path 'script-languages'
Cloning into '/home/ubuntu/notebooks/script-languages-release/script-languages'...
remote: Enumerating objects: 8453, done.        
remote: Counting objects: 100% (1014/1014), done.        
remote: Compressing objects: 100% (527/527), done.        
remote: Total 8453 (delta 516), reused 835 (delta 446), pack-reused 7439        
Receiving objects: 100% (8453/8453), 19.25 MiB | 22.40 MiB/s, done.
Resolving deltas: 100% (4790/4790), done.
Submodule path 'script-languages': checked out '0bd87605635d5417ab9d0aeeb7f

## Buiding and Exporting a Container

To build and export the container you can use `exaslct`. It first builds a series of Docker images and then exports the container as a `tar.gz` package. We provide several flavors of containers with different capabilities. You can find out more about the flavors in our [flavor documentation on Github.](https://github.com/exasol/script-languages-release/blob/master/flavors/README.md). Flavors are described by a flavor definition in the directory `flavors/`. Here is an overview of the available flavors:

In [8]:
bash.run(f"""
find {slc_path}/flavors/  -maxdepth 1 -name '*EXASOL*'
""")

script-languages-release/flavors/python-3.7-minimal-EXASOL-6.2.0
script-languages-release/flavors/standard-EXASOL-8.0.0
script-languages-release/flavors/standard-EXASOL-7.0.0
script-languages-release/flavors/r-4-minimal-EXASOL-6.2.0
script-languages-release/flavors/python-3.6-minimal-EXASOL-6.2.0
script-languages-release/flavors/python-3.8-minimal-EXASOL-6.2.0
script-languages-release/flavors/standard-EXASOL-7.1.0
script-languages-release/flavors/standard-EXASOL-7.1.0-without-python2.7


For this example, we use the `flavors/python-3.8-minimal-EXASOL-6.2.0` flavor and export it to the `containers` directory.

A container gets built via a series of Docker images, and then it's exported into a `tar` file.

In [9]:
bash.run(f"""
pushd {slc_path}
./exaslct export --flavor-path flavors/python-3.8-minimal-EXASOL-6.2.0 --export-path containers
""")

~/notebooks/script-languages-release ~/notebooks
container-tool-runner-0.15.0: Pulling from exasol/script-language-container

ec22a9e9: Pulling fs layer 
dc18882e: Pulling fs layer 
ba24f29a: Pulling fs layer 
888ef4ae: Pulling fs layer 
38f67598: Pulling fs layer 
f06dbf7e: Pulling fs layer 
cad4f063: Pulling fs layer 
9df25eb9: Pulling fs layer 
9ff3892d: Pulling fs layer 
8642517d: Pulling fs layer 
e751e299: Pulling fs layer 
5d452c69: Pulling fs layer 
Digest: sha256:b623e78bd3784908965c3dfe14351d0a8c3c9741f4edd8110174f926281341c2
Status: Downloaded newer image for exasol/script-language-container:container-tool-runner-0.15.0
docker.io/exasol/script-language-container:container-tool-runner-0.15.0
The command took 105.041045 s

Cached container under /home/ubuntu/notebooks/script-languages-release/.build_output/cache/exports/python-3.8-minimal-EXASOL-6.2.0-release-ZOLA663JXOLQ6D3BGWBRLRI3G57HAHQGGU2CUYISFNDSPMJAO23Q.tar.gz

Copied container to containers/python-3.8-minimal-EXASOL-6

### What to do if something doesn't work?

During the build it can happen that external package repositories might not be available or something is wrong on your machine where you run the build. For these cases, `exaslct` stores many logs to identify the problem.

#### Exaslsct Log

The main log for `exaslct` is stored directly as `main.log` in the build output of the job. With the following command you can find the main logs for all previous executions.

In [13]:
bash.run(f"""
pushd {slc_path}
find .build_output -name 'main.log'
""")

~/notebooks/script-languages-release ~/notebooks
.build_output/jobs/logs/main.log


With the following command you can show the log file from the last execution.

In [14]:
bash.run(f"""
pushd {slc_path}
LAST_LOG="$(find .build_output -name 'main.log' | sort |tail -n 1)"
cat $LAST_LOG | tail -n 20
""")

~/notebooks/script-languages-release ~/notebooks
2022-09-22 14:25:28,866 - luigi-interface.AnalyzeRelease - DEBUG - AnalyzeRelease_a35e3e0aa1(job_id=2022_09_22_14_23_43_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Cleaning up
2022-09-22 14:25:28,867 - luigi-interface.AnalyzeLanguageDeps - DEBUG - AnalyzeLanguageDeps_a35e3e0aa1(job_id=2022_09_22_14_23_43_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Cleaning up
2022-09-22 14:25:28,867 - luigi-interface.AnalyzeUDFClientDeps - DEBUG - AnalyzeUDFClientDeps_a35e3e0aa1(job_id=2022_09_22_14_23_43_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Cleaning up
2022-09-22 14:25:28,869 - luigi-interface.AnalyzeUDFClientDeps - DEBUG - AnalyzeUDFClientDeps_a35e3e0aa1(job_id=2022_09_22_14_23_43_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Cleanup finished
2022-09-22 14:25:28,870 - luigi-

#### Build Output Directory

More detailed information about the build or other operations can be found in the `.build_output/jobs/*/outputs` directory. Here each run of `exaslct` creates its own directory under `.build_output/jobs`. The `outputs` directory stores the outputs and log files (if any) that each executed task of `exaslct` produces. Especially, the Docker tasks such as build, pull and push store the logs returned by the Docker API. This can be helpful for finding problems during the build.

In [15]:
bash.run(f"""
pushd {slc_path}
find .build_output/jobs/*/outputs -type f -name '*log' | tail
""")

~/notebooks/script-languages-release ~/notebooks
.build_output/jobs/2022_09_22_14_23_43_ExportContainers/outputs/ExportContainers_e31c9ee4d3/ExportFlavorContainer_ce7a119768/ExportContainerTask_17ac1632df/logs/extract_release_file.log
.build_output/jobs/2022_09_22_14_23_43_ExportContainers/outputs/ExportContainers_e31c9ee4d3/ExportFlavorContainer_ce7a119768/ExportContainerTask_17ac1632df/logs/pack_release_file.log
.build_output/jobs/2022_09_22_14_23_43_ExportContainers/outputs/ExportContainers_e31c9ee4d3/ExportFlavorContainer_ce7a119768/DockerCreateImageTask_0062081224/DockerPullImageTask_0062081224/logs/pull_docker_db_image.log


## Customizing Script-Language Containers

Sometimes you need very specific dependencies or versions of dependencies in the Exasol UDFs. In such case you can customize a Script-Language Container.

### Flavor Definition

To customize a flavor, you need to change the flavor definition. A flavor consists of several build steps. The following images gives you an idea about how these build steps are connected with each other.

![Flavor Structure](slc_main_build_steps.svg)

For customizing a flavor usually the `flavor_customization` build step is most important. It contains everything you need to add dependencies. The remaining build steps should be only changed with care, but sometimes some dependencies are defined in other build steps because the script client depends on them. Here you see the directory structure of the flavor definition for our example flavor `flavors/python-3.8-minimal-EXASOL-6.2.0`.

In [16]:
bash.run(f""" 
pushd {slc_path}
find -L flavors/python-3.8-minimal-EXASOL-6.2.0 -maxdepth 2
""")

~/notebooks/script-languages-release ~/notebooks
flavors/python-3.8-minimal-EXASOL-6.2.0
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/Dockerfile
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages
flavors/python-3.8-minimal-EXASOL-6.2.0/FLAVOR_DESCRIPTION.md
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/base_test_deps
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/build_run
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/build_steps.py
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/__pycache__
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/release
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/build_deps
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/base_test_build_run
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/security_scan
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/flavor_base_deps
flav

### Flavor Customization Build Step

The `flavor_customization` build step consists of a Dockerfile and several package lists which can be modified. We recommend to use the package lists to add new packages to the flavor and only modify the Dockerfile if you need very specific changes, like adding additional resources.

In [17]:
bash.run(f""" 
pushd {slc_path}
find -L flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization -type f
""")

~/notebooks/script-languages-release ~/notebooks
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/Dockerfile
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/apt_get_packages


#### Dockerfile

The Dockerfile consists of two parts. The first part installs the packages from the package list and should only be change with care. The second part is free for your changes. Read the description in the Dockerfile carefully to find out what you can and shouldn't do.

In [18]:
bash.run(f""" 
pushd {slc_path}
cat flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/Dockerfile
""")

~/notebooks/script-languages-release ~/notebooks
############################################################################################
############################################################################################
# This Dockerfile allows you to extend this flavor by installing packages or adding files. 
# IF you didn't change the lines below, you can add packages and their version to the  
# files in ./packages and they get automatically installed.                                
############################################################################################
############################################################################################

#######################################################################
#######################################################################
# Do not change the following lines unless you know what you are doing 
#######################################################################
##################

#### Package Lists

The package lists have a unified format. Each line consists of the package name and the package version separated by "|", e.g `xgboost|1.3.3`. You can comment out a whole line by adding"#" at the beginning. You can also add a trailing comment to a package definition by adding a "#" after the package definition. We usually recommend to pin the version, such that there are no surprises for which version gets installed.

In [19]:
bash.run(f""" 
pushd {slc_path}
cat flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
""")

~/notebooks/script-languages-release ~/notebooks
# This file specifies the package list which gets installed via pip for python3.
# You must specify the the package and its version separated by a |.
# We recommend here the usage of package versions, to ensure that the container 
# builds are reproducible. However, we allow also packages without version.
# As you can see, this file can contain comments which start with #.
# If a line starts with # the whole line is a comment, however you can
# also start a comment after the package definition.

#tensorflow-probability|0.9.0


We are now going to append the "xgboost" Python package to one of the package lists by adding `"xgboost|1.3.3"` and `scikit-learn|0.24.2` to the `flavor_customization/packages/python3_pip_packages` file.

In [20]:
bash.run(f""" 
pushd {slc_path}
echo "xgboost|1.3.3" > flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
echo "scikit-learn|0.24.2" >> flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
""")

~/notebooks/script-languages-release ~/notebooks


As you can see below, we now added a new line to our package list.

In [21]:
bash.run(f""" 
pushd {slc_path}
cat flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
""")

~/notebooks/script-languages-release ~/notebooks
xgboost|1.3.3
scikit-learn|0.24.2


#### Rebuilding the customized Flavor

After changing the flavor you need to rebuild it. You can do it by running `./exaslsct export --flavor-path <flavor-path>` again. Exaslct automatically recognizes that the flavor has changed and builds a new version of the container.

In [22]:
bash.run(f"""
pushd {slc_path}
./exaslct export --flavor-path flavors/python-3.8-minimal-EXASOL-6.2.0 --export-path containers
""")

~/notebooks/script-languages-release ~/notebooks
WARNING - AnalyzeFlavorCustomization_96aa0fe2f5(job_id=2022_09_22_14_39_44_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Image exasol/script-language-container:python-3.8-minimal-EXASOL-6.2.0-flavor_customization_KKTMMDEYGMN4BRMA6IICTIWXFGIIHWN3RUU3N4DJOL5P3OU6O5NQ not in registry, got exception 404 Client Error for http+docker://localhost/v1.41/images/create?tag=python-3.8-minimal-EXASOL-6.2.0-flavor_customization_KKTMMDEYGMN4BRMA6IICTIWXFGIIHWN3RUU3N4DJOL5P3OU6O5NQ&fromImage=exasol%2Fscript-language-container: Not Found ("manifest for exasol/script-language-container:python-3.8-minimal-EXASOL-6.2.0-flavor_customization_KKTMMDEYGMN4BRMA6IICTIWXFGIIHWN3RUU3N4DJOL5P3OU6O5NQ not found: manifest unknown: manifest unknown")
WARNING - AnalyzeRelease_96aa0fe2f5(job_id=2022_09_22_14_39_44_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Image exasol/script-language

**Note:** Your old container doesn't get lost, because when you change a flavor your container gets a new hash code. If you revert your changes the system automatically uses the existing cached container. Below you can see the content of the cache directory for the containers.

In [23]:
bash.run(f"""
pushd {slc_path}
ls -sh .build_output/cache/exports
""")

~/notebooks/script-languages-release ~/notebooks
total 611M
417M python-3.8-minimal-EXASOL-6.2.0-release-3WFA2GJTKANDAEI5T44F5ON7EWWJX2JPWMRZ3NGD4D7QBU3EKGXQ.tar.gz
4.0K python-3.8-minimal-EXASOL-6.2.0-release-3WFA2GJTKANDAEI5T44F5ON7EWWJX2JPWMRZ3NGD4D7QBU3EKGXQ.tar.gz.sha512sum
195M python-3.8-minimal-EXASOL-6.2.0-release-ZOLA663JXOLQ6D3BGWBRLRI3G57HAHQGGU2CUYISFNDSPMJAO23Q.tar.gz
4.0K python-3.8-minimal-EXASOL-6.2.0-release-ZOLA663JXOLQ6D3BGWBRLRI3G57HAHQGGU2CUYISFNDSPMJAO23Q.tar.gz.sha512sum


## Testing the new Script-Language Container

Now, that we have an updated container, we need to check if our changes were successful. For that we are going to upload the container to an Exasol Database and have a look into it. In this example, we are going to use a local Docker-DB started by `exaslct`, which uses our [integration-test-docker-environment](https://github.com/exasol/integration-test-docker-environment) in the background. 

**Note:** You could also use your own Exasol Database by changing the variables below. However, this Notebook must be able to access the BucketFS of your Exasol Database or you need to manually upload the container. 

In [26]:
DATABASE_HOST="localhost"
DATABASE_PORT=8886
DATABASE_USER="sys"
DATABASE_PASSWORD="exasol"
BUCKETFS_PORT=6666
BUCKETFS_USER="w"
BUCKETFS_PASSWORD="write"
BUCKETFS_NAME="bfsdefault"
BUCKET_NAME="default"
PATH_IN_BUCKET="container"

### Starting a local Docker-DB for Testing

The exaslct command also allows us to create rapidly a test environment where we can test the new container!

#### Start the environment and forward the database and bucketfs ports to the specified host ports.

**Note:** The Exasol Integration-Test-Docker-Environment requires Docker with privileged mode

**Note:** Starting the environment can take between 3-5 min.

In [27]:
bash.run(f"""
pushd {slc_path}
./exaslct spawn-test-environment --environment-name test --database-port-forward {DATABASE_PORT} --bucketfs-port-forward {BUCKETFS_PORT}
""")

~/notebooks/script-languages-release ~/notebooks
WARNING - SpawnTestContainer_203f5cf295(job_id=2022_09_22_15_03_54_SpawnTestEnvironmentWithDockerDB, no_cache=False, environment_name=test, test_container_name=test_container_test, attempt=0): Copy tests in test container test_container_test.
WARNING - PopulateEngineSmallTestDataToDatabase_8816acdffe(job_id=2022_09_22_15_03_54_SpawnTestEnvironmentWithDockerDB, no_cache=False, db_user=sys, environment_name=test): Uploading data
The command took 235.088755 s


### Upload the Container to the Database

To use our container we need to upload it to the BucketFS . If the build machine has access to the BucketFS we can do it with the `exaslct upload` command, otherwise you need to export the container and transfer it to a machine that has access to the BucketFS and upload it via `curl`, as described in our [documentation](https://docs.exasol.com/database_concepts/udf_scripts/adding_new_packages_script_languages.htm).

With the following command we upload the new script language container to the previously created test environment.

In [28]:
bash.run(f"""
pushd {slc_path}
./exaslct upload \
    --flavor-path flavors/python-3.8-minimal-EXASOL-6.2.0 \
    --database-host {DATABASE_HOST}\
    --bucketfs-port {BUCKETFS_PORT} \
    --bucketfs-username {BUCKETFS_USER} \
    --bucketfs-password {BUCKETFS_PASSWORD} \
    --bucketfs-name {BUCKETFS_NAME} \
    --bucket-name {BUCKET_NAME} \
    --path-in-bucket {PATH_IN_BUCKET} \
    --release-name current
""")

~/notebooks/script-languages-release ~/notebooks
The command took 84.778999 s

Uploaded .build_output/cache/exports/python-3.8-minimal-EXASOL-6.2.0-release-3WFA2GJTKANDAEI5T44F5ON7EWWJX2JPWMRZ3NGD4D7QBU3EKGXQ.tar.gz to
http://localhost:6666/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current.tar.gz


In SQL, you can activate the languages supported by the python-3.8-minimal-EXASOL-6.2.0
flavor by using the following statements:


To activate the flavor only for the current session:

ALTER SESSION SET SCRIPT_LANGUAGES='PYTHON3=localzmq+protobuf:///bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current?lang=python#buckets/bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current/exaudf/exaudfclient_py3';


To activate the flavor on the system:

ALTER SYSTEM SET SCRIPT_LANGUAGES='PYTHON3=localzmq+protobuf:///bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current?lang=python#buckets/bfsdefault/default/container/py

### Getting the language container activation statement without upload

Sometimes you can't use the `upload` command to upload your container to the BucketFS. To get the language activation statement regardless of that, you can use the `generate-language-activation` command.

In [29]:
bash.run(f"""
pushd {slc_path}
./exaslct generate-language-activation \
    --flavor-path flavors/python-3.8-minimal-EXASOL-6.2.0 \
    --bucketfs-name {BUCKETFS_NAME} \
    --bucket-name {BUCKET_NAME} \
    --path-in-bucket {PATH_IN_BUCKET} \
    --container-name python-3.8-minimal-EXASOL-6.2.0-current  2>&1 | tail -n 15
""")

~/notebooks/script-languages-release ~/notebooks


In SQL, you can activate the languages supported by the python-3.8-minimal-EXASOL-6.2.0
flavor by using the following statements:


To activate the flavor only for the current session:

ALTER SESSION SET SCRIPT_LANGUAGES='PYTHON3=localzmq+protobuf:///bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-current?lang=python#buckets/bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-current/exaudf/exaudfclient_py3';


To activate the flavor on the system:

ALTER SYSTEM SET SCRIPT_LANGUAGES='PYTHON3=localzmq+protobuf:///bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-current?lang=python#buckets/bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-current/exaudf/exaudfclient_py3';



### Connecting to the database and activate the container

Once we have a connection to the database we run the `ALTER SESSION` statement or `ALTER SYSTEM` statement (if you want to activate the container permanently and globally) we got from the upload.

In [30]:
def connect():
    con=pyexasol.connect(dsn=f"{DATABASE_HOST}:{DATABASE_PORT}",user=DATABASE_USER,password=DATABASE_PASSWORD)
    con.execute("ALTER SESSION SET SCRIPT_LANGUAGES='PYTHON3=builtin_python3 PYTHON3_DS=localzmq+protobuf:///bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current?lang=python#buckets/bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current/exaudf/exaudfclient_py3';")
    con.execute("OPEN SCHEMA TEST")
    return con

### Check if your customization did work

We first create a helper UDF which allows us to run arbitrary shell commands inside of a UDF instance. With that we can easily inspect the container.

In [31]:
con = connect()

con.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3_DS SCALAR SCRIPT execute_shell_command_py3(command VARCHAR(2000000), split_output boolean)
EMITS (lines VARCHAR(2000000)) AS
import subprocess

def run(ctx):
    try:
        p = subprocess.Popen(ctx.command,
                             stdout    = subprocess.PIPE,
                             stderr    = subprocess.STDOUT,
                             close_fds = True,
                             shell     = True)
        out, err = p.communicate()
        if isinstance(out,bytes):
            out=out.decode('utf8')
        if ctx.split_output:
            for line in out.strip().split('\\n'):
                ctx.emit(line)
        else:
            ctx.emit(out)
    finally:
        if p is not None:
            try: p.kill()
            except: pass
/
"""))

<ExaStatement session_id=1744683306169139200 stmt_idx=3>

#### Check with "pip list" if the  "xgboost" package is installed

We use our helper UDF to run `python3 -m pip list` directly in the container and get the list of currently avaiable python3 packages.

In [32]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('python3 -m pip list', true)""")
for r in rs: 
    print(r[0])

Package         Version
--------------- --------
joblib          1.2.0
numpy           1.21.3
pandas          1.3.4
pip             21.3.1
python-dateutil 2.8.2
pytz            2022.2.1
scikit-learn    0.24.2
scipy           1.9.1
setuptools      65.3.0
six             1.16.0
threadpoolctl   3.1.0
wheel           0.37.1
xgboost         1.3.3


By running `pip list` directly in the container, we see what is currently available in the container. However, sometimes this might not be what we expected. For these cases, `exaslct` stores information about the flavor the container was build from within the container.

#### Embedded Build Information of the Container

Here we see an overview about the build information which `exaslct` embedded into the container. `Exaslct` stores all packages lists (as defined in the flavor and what actually got installed), the final Dockerfiles and the image info. The image info describes how the underlying Docker images of the container got built. The build information is stored in the `/build_info` directory in the container. We can use again our helper UDF to inspect the build information.

In [33]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('find /build_info', true)""")
for r in rs: 
    print(r[0])

/build_info
/build_info/packages
/build_info/packages/flavor_customization
/build_info/packages/flavor_customization/python3_pip_packages
/build_info/packages/flavor_customization/apt_get_packages
/build_info/packages/build_deps
/build_info/packages/build_deps/apt_get_packages
/build_info/packages/flavor_base_deps
/build_info/packages/flavor_base_deps/python3_pip_packages
/build_info/packages/flavor_base_deps/apt_get_packages
/build_info/packages/udfclient_deps
/build_info/packages/udfclient_deps/apt_get_packages
/build_info/packages/language_deps
/build_info/packages/language_deps/python3_pip_packages
/build_info/packages/language_deps/apt_get_packages
/build_info/actual_installed_packages
/build_info/actual_installed_packages/release
/build_info/actual_installed_packages/release/python3_pip_packages
/build_info/actual_installed_packages/release/apt_get_packages
/build_info/image_info
/build_info/image_info/python-3.8-minimal-EXASOL-6.2.0-language_deps
/build_info/image_info/python-3.

The following command shows for example, which python3 package pip found directly after the build of the container image.

In [34]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('cat /build_info/actual_installed_packages/release/python3_pip_packages', true)""")
for r in rs: 
    print(r[0])

joblib|1.2.0
numpy|1.21.3
pandas|1.3.4
pip|21.3.1
python-dateutil|2.8.2
pytz|2022.2.1
scikit-learn|0.24.2
scipy|1.9.1
setuptools|65.3.0
six|1.16.0
threadpoolctl|3.1.0
wheel|0.37.1
xgboost|1.3.3


You could for example compare this to the package list of the `flavor-customization` build step and check if all your requested packages got installed.

In [35]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('cat /build_info/packages/flavor_customization/python3_pip_packages', true)""")
for r in rs:
    if r[0] is None:
        print()
    else:
        print(r[0])

xgboost|1.3.3
scikit-learn|0.24.2


### Testing the new package

After we made sure that the required packages are installed, we need to try importing and using it. Importing is usually a good first test if a package got successfully installed, because often you might already get errors at this step. However, sometimes you only will recognize errors when using the package. We recommend to have a test suite for each new package to check if it works properly before you start your UDF development. It is usually easier to debug problems if you have very narrow tests.

In [36]:
con = connect()

con.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3_DS SET SCRIPT test_xgboost(i integer)
EMITS (o VARCHAR(2000000)) AS

def run(ctx):
    import xgboost
    import sklearn 
    
    ctx.emit("success")
/
"""))

rs = con.execute("select test_xgboost(1)")
rs.fetchall()

[('success',)]

In [37]:
con = connect()

con.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3_DS SET SCRIPT test_xgboost(i integer)
EMITS (o1 DOUbLE, o2 DOUbLE, o3 DOUbLE) AS

def run(ctx):
    import pandas as pd
    import xgboost as xgb
    from sklearn import datasets
    from sklearn.model_selection import train_test_split
    
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    param = {
        'max_depth': 3,  # the maximum depth of each tree
        'eta': 0.3,  # the training step for each iteration
        'silent': 1,  # logging mode - quiet
        'objective': 'multi:softprob',  # error evaluation for multiclass training
        'num_class': 3  # the number of classes that exist in this datset
        }
    num_round = 20  # the number of training iterations
    bst = xgb.train(param, dtrain, num_round)
    preds = bst.predict(dtest)
    
    ctx.emit(pd.DataFrame(preds))
/
"""))

con.export_to_pandas("select test_xgboost(1)")

,O1,O2,O3
0,0.005638,0.977552,0.016810
1,0.982547,0.013958,0.003495
2,0.003637,0.006152,0.990210
3,0.005647,0.979170,0.015182
4,0.005401,0.936409,0.058190
5,0.986080,0.010413,0.003508
6,0.004390,0.990413,0.005198
7,0.015695,0.066531,0.917774
8,0.006338,0.948772,0.044891
9,0.004390,0.990413,0.005198
